In [1]:
import numpy as np
import pandas as pd
import math


In [3]:
 #Map values in DF
column_mappings = {
    'age': {'<=30': 0, '31 to 40': 1, '>40': 2},
    'income': {'low': 0, 'medium': 1, 'high': 2},
    'student': {'no': 0, 'yes': 1},
    'credit_rating': {'fair': 0, 'excellent': 1},
    'buys_computer': {'no': 0, 'yes': 1}
}

# dataset
data = {
    'age': ['<=30', '<=30', '31 to 40', '>40', '>40', '>40', '31 to 40', '<=30', '<=30', '>40', '<=30', '31 to 40', '31 to 40', '>40'],
    'income': ['high', 'high', 'high', 'medium', 'low', 'low', 'low', 'medium', 'low', 'medium', 'medium', 'medium', 'high', 'medium'],
    'student': ['no', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no'],
    'credit_rating': ['fair', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'fair', 'fair', 'excellent', 'excellent', 'fair', 'excellent'],
    'buys_computer': ['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']
}

In [4]:
df = pd.DataFrame(data)

# Encode categorical variables
categorical_columns = ['age', 'income', 'student', 'credit_rating', 'buys_computer']
df.replace(column_mappings, inplace=True)


In [5]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y, depth=0):
        if depth == self.max_depth or len(set(y)) == 1:

            unique_labels, counts = np.unique(y, return_counts=True)
            return {'class': unique_labels[np.argmax(counts)]}

        best_feature, best_threshold = self.find_best_split(X, y)

        if best_feature is None:

            unique_labels, counts = np.unique(y, return_counts=True)
            return {'class': unique_labels[np.argmax(counts)]}

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = ~left_indices


        left_subtree = self.fit(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self.fit(X[right_indices], y[right_indices], depth + 1)

        return {
            'feature_index': best_feature,
            'threshold': best_threshold,
            'left': left_subtree,
            'right': right_subtree
        }

    def find_best_split(self, X, y):
        num_features = X.shape[1]
        best_feature = None
        best_threshold = None
        best_info_gain = 0

        for feature_index in range(num_features):
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                left_indices = X[:, feature_index] <= threshold
                right_indices = ~left_indices

                if len(y[left_indices]) == 0 or len(y[right_indices]) == 0:
                    continue

                info_gain = self.calculate_info_gain(y, y[left_indices], y[right_indices])

                if info_gain > best_info_gain:
                    best_info_gain = info_gain
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold

    def calculate_info_gain(self, parent, left_child, right_child):
        parent_entropy = self.entropy(parent)
        left_child_entropy = (len(left_child) / len(parent)) * self.entropy(left_child)
        right_child_entropy = (len(right_child) / len(parent)) * self.entropy(right_child)

        info_gain = parent_entropy - (left_child_entropy + right_child_entropy)
        return info_gain

    def entropy(self, labels):

        label_counts = {}
        for label in labels:
            if label in label_counts:
                label_counts[label] += 1
            else:
                label_counts[label] = 1


        entropy_value = 0
        total_instances = len(labels)
        for count in label_counts.values():
            probability = count / total_instances
            entropy_value -= probability * math.log2(probability)

        return entropy_value

    def predict_instance(self, instance, tree):
        if 'class' in tree:
            return tree['class']

        if instance[tree['feature_index']] <= tree['threshold']:
            return self.predict_instance(instance, tree['left'])
        else:
            return self.predict_instance(instance, tree['right'])

    def predict(self, X, tree):
        return [self.predict_instance(instance, tree) for instance in X]

    def visualize_tree(self, tree, indent=''):
        if 'class' in tree:
            print(f"{indent}Class: {tree['class']}")
        else:
            print(f"{indent}Feature {tree['feature_index']} <= {tree['threshold']}")
            print(f"{indent}True:")
            self.visualize_tree(tree['left'], indent + '  ')
            print(f"{indent}False:")
            self.visualize_tree(tree['right'], indent + '  ')


In [10]:
X = np.array(df[['age', 'income', 'student', 'credit_rating']])
y = np.array(df['buys_computer'])

tree = DecisionTree(max_depth=6)
model = tree.fit(X, y)

In [11]:
X_test = np.array([['>40', 'low', 'no', 'fair'], ['<=30', 'low', 'yes', 'fair']])
# Convert the test dataset in the same way as the training dataset
X_test = np.array([[column_mappings['age'][X_test[0][0]], column_mappings['income'][X_test[0][1]],
                    column_mappings['student'][X_test[0][2]], column_mappings['credit_rating'][X_test[0][3]]],
                   [column_mappings['age'][X_test[1][0]], column_mappings['income'][X_test[1][1]],
                    column_mappings['student'][X_test[1][2]], column_mappings['credit_rating'][X_test[1][3]]]])
print(X_test)
predictions = tree.predict(X_test, model)

print("Predictions:", predictions)


[[2 0 0 0]
 [0 0 1 0]]
Predictions: [1, 1]


In [12]:
tree.visualize_tree(model)


Feature 2 <= 0
True:
  Feature 0 <= 0
  True:
    Class: 0
  False:
    Feature 0 <= 1
    True:
      Class: 1
    False:
      Feature 3 <= 0
      True:
        Class: 1
      False:
        Class: 0
False:
  Feature 0 <= 1
  True:
    Class: 1
  False:
    Feature 3 <= 0
    True:
      Class: 1
    False:
      Class: 0
